In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_iters = 250

In [3]:

with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
#this will create a list of all the characters in the text file
#the set function will remove all the duplicates
#the sorted function will sort the characters in the list
   #use a tokenizer to convert each element into  an integer

    #

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [5]:

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[67, 58, 57,  1, 73, 68,  1, 73],
        [67,  1, 54,  1, 55, 54, 65, 65],
        [62, 64, 58,  1, 55, 58, 62, 67],
        [74, 72, 58, 57,  9,  1, 54, 67]])
targets:
tensor([[58, 57,  1, 73, 68,  1, 73, 61],
        [ 1, 54,  1, 55, 54, 65, 65, 68],
        [64, 58,  1, 55, 58, 62, 67, 60],
        [72, 58, 57,  9,  1, 54, 67, 57]])


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [7]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?;aVfI cXICL&YZ8J'vd&&.!H-s1)0HmQ,.TV&7nk9A_Lb0i?6i]0x?y6sv&T!W*"0U!_x]4fIeyAVBj"﻿Q,U!&_8e!HOeEvU gi]- D)j(x:]!VJfcB﻿Yz
?L8BBFFN&bSOwhYA'n﻿z3V&p-2mC]KunBr[ID2UB3uhf0YOOACg6!mq](PL-F:9XWB7n?LAmm'1kIqGCi?L8?tx:4[AUOeW((P-[A3(QOAqcR]5.YN&TfI7&CM-HdCurk]LqB,am&tF, ,1stL9k?L;W(t7UE Y0g;yI7z﻿bS T['o?:4A4o?1i*O.y"_7qB;QDu"3TKPqc7bSucDE87pbuyenw'rk;-ny".S OI
y[_I﻿z-H5OiZ(ai"06E)6BMdWCuTg;GC0wSjUL1'"﻿a'U5x?hv25&0kpiQDyc:,Q?LOEPy[1qhvI Vf,﻿;vvSkD&_kAMG4'fO'oxc-z6wZZ4y(zq;J9"﻿r8d'djsIbS!*7&Gz2IbS_hAS 9H5Lb


In [8]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.900, val loss: 4.913
step: 250, train loss: 4.313, val loss: 4.334
step: 500, train loss: 3.846, val loss: 3.855
step: 750, train loss: 3.486, val loss: 3.502
3.4114227294921875
